In [ ]:
class SwarmGenerator:

  def __init__(self,objective_function,n_iteration=50,population_size=50,
               c1=2,c2=2,w=0.9):
    self.population_size = population_size
    self.n_iteration=n_iteration
    self.historical_score_record=[]
    self.objective_function=objective_function
    self.c1=c1
    self.c2=c2
    self.w=w

  def initialize_population(self,X):
        self.individuals =  np.random.randint(0,2,size=(self.population_size,X.shape[1]))


  def evaluate_fitness(self,model,X_train,y_train,X_valid,y_valid):
        scores =  []
        for i,individual in enumerate(self.individuals):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.current_best_scores[i]:
              self.current_best_scores[i]=score
              self.current_best_individual_score_dimensions[i]=individual

            scores.append(score)
        self.fitness_scores = scores
 

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))
        
  def fit(self,model,X_train,y_train,X_valid,y_valid):
    list_best_fitness=[]
    list_mean_fitness=[]
    list_min_fitness=[]

    self.initialize_population(X_train)

    self.best_score=np.inf
    self.best_dim=np.ones(X_train.shape[1]) 

    self.current_best_individual_score_dimensions=self.individuals
    self.current_best_scores = [np.inf]*self.population_size 
    self.gbest_individual=self.best_dim
    self.v=np.zeros((self.population_size,X_train.shape[1]))

    for i in range(self.n_iteration):
      if (self.individuals.sum(axis=1)==0).sum()>0:
        print((self.individuals.sum(axis=1)==0).sum(),' individuals went zero')
        self.individuals[self.individuals.sum(axis=1)==0]=np.random.randint(0,2,(self.individuals[self.individuals.sum(axis=1)==0].shape[0],\
                                                                                self.individuals[self.individuals.sum(axis=1)==0].shape[1]))
      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)

      if np.array(self.fitness_scores).min()<self.best_score:
        self.best_dim=self.individuals[np.array(self.fitness_scores).argmin()]
        self.best_score=np.array(self.fitness_scores).min()
      self.gbest_individual=self.best_dim       

      r1=np.random.random((self.population_size,X_train.shape[1]))
      r2=np.random.random((self.population_size,X_train.shape[1]))
    
      self.v=self.w*self.v+self.c1*r1*(self.gbest_individual-self.individuals)+\
      self.c2*r2*(self.current_best_individual_score_dimensions-self.individuals)
      self.v=np.where(self.v>6,6,self.v)
      self.v=np.where(self.v<-6,-6,self.v)

      list_best_fitness.append(self.best_score)
      list_mean_fitness.append(np.array(self.fitness_scores).mean())
      list_min_fitness.append(np.array(self.fitness_scores).min())
      
      
      self.s_v=self.sigmoid(self.v)

      self.individuals=np.where(np.random.uniform(size=(self.population_size,X_train.shape[1]))<self.s_v,1,0)

    return self.best_dim,   list_best_fitness,\
                list_mean_fitness,list_min_fitness

gaf=SwarmGenerator(score_model,100,10)
res,temp1,temp2,temp3=gaf.fit(clf,X_train, y_train,X_test, y_test)